In [1]:
import os
from random import random

import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split

from datetime import datetime

In [2]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [3]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [5]:
df_train = image_processor.process_image_data_with_edges("data/Train", image_size, targets=df_names)

In [6]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [7]:
X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.005, stratify=y_data, random_state=42)

In [9]:
model = model_builder.build_conv_pool_model(input_size, len(df_names))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=256,
                    callbacks=[early_stopping, tensorboard_callback],
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/50
207/207 [==============================] - 21s 100ms/step - loss: 1.4424 - accuracy: 0.6089 - val_loss: 3.5549 - val_accuracy: 0.1090
Epoch 2/50
207/207 [==============================] - 18s 88ms/step - loss: 0.5938 - accuracy: 0.8085 - val_loss: 0.6883 - val_accuracy: 0.8120
Epoch 3/50
207/207 [==============================] - 18s 87ms/step - loss: 0.4631 - accuracy: 0.8415 - val_loss: 0.3597 - val_accuracy: 0.8684
Epoch 4/50
207/207 [==============================] - 16s 78ms/step - loss: 0.3971 - accuracy: 0.8570 - val_loss: 0.3150 - val_accuracy: 0.8947
Epoch 5/50
207/207 [==============================] - 16s 79ms/step - loss: 0.3535 - accuracy: 0.8680 - val_loss: 0.2832 - val_accuracy: 0.9098
Epoch 6/50
207/207 [==============================] - 17s 82ms/step - loss: 0.3144 - accuracy: 0.8807 - val_loss: 0.2919 - val_accuracy: 0.8835
Epoch 7/50
207/207 [==============================] - 16s 78ms/step - loss: 0.2909 - accuracy: 0.8865 - val_loss: 0.2852 - val_accuracy

In [10]:
for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
    print(f"Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

score = model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {score[1] * 100:.2f}%")
# 91.84

Epoch 1: val_loss=3.5549, val_accuracy=0.1090
Epoch 2: val_loss=0.6883, val_accuracy=0.8120
Epoch 3: val_loss=0.3597, val_accuracy=0.8684
Epoch 4: val_loss=0.3150, val_accuracy=0.8947
Epoch 5: val_loss=0.2832, val_accuracy=0.9098
Epoch 6: val_loss=0.2919, val_accuracy=0.8835
Epoch 7: val_loss=0.2852, val_accuracy=0.8910
Epoch 8: val_loss=0.2886, val_accuracy=0.9023
Epoch 9: val_loss=0.2581, val_accuracy=0.8947
Epoch 10: val_loss=0.2866, val_accuracy=0.9023
Epoch 11: val_loss=0.2443, val_accuracy=0.9098
Epoch 12: val_loss=0.2492, val_accuracy=0.9173
Epoch 13: val_loss=0.2623, val_accuracy=0.9248
Epoch 14: val_loss=0.2549, val_accuracy=0.9211
Epoch 15: val_loss=0.2872, val_accuracy=0.9098
Epoch 16: val_loss=0.2317, val_accuracy=0.9173
Epoch 17: val_loss=0.2984, val_accuracy=0.9023
Epoch 18: val_loss=0.2862, val_accuracy=0.9023
Epoch 19: val_loss=0.2696, val_accuracy=0.9135
Epoch 20: val_loss=0.2807, val_accuracy=0.8985
Epoch 21: val_loss=0.3089, val_accuracy=0.9173
Final Validation Accur

In [13]:
test_df = df_test = image_processor.process_image_data("data/TestData", image_size)

In [14]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

pred = model.predict(X_test_prepared)
predicted_classes = pred.argmax(axis=1)
predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_labels

222/222 [==============================] - 1s 5ms/step


In [15]:
model.save("models/model_92_05")

INFO:tensorflow:Assets written to: models/model_92_05/assets


INFO:tensorflow:Assets written to: models/model_92_05/assets


In [17]:
filtered_df = test_df[test_df['pred'] == '']
test_df[['label', 'pred']]

,label,pred
0,Test4751.png,2
1,Test2320.png,k
2,Test4989.png,N
3,Test6146.png,S
4,Test1629.png,s
...,...,...
7095,Test2339.png,p
7096,Test4990.png,N
7097,Test5456.png,9
7098,Test4748.png,0


In [ ]:
filtered_df = test_df[test_df['label'] == 'Test0294.png']